In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import os
from glob import glob
from collections import OrderedDict
import xarray as xr
import numpy as np
import dask
from ncar_jobqueue import NCARCluster
import esmlab
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs
from scipy import stats
import pop_tools 

from statistics import mean
from matplotlib import style
from scipy import stats
style.use('ggplot')

#### Diagnostics for run g.e21.G1850ECOIAF.T62_g16.003:
Difference from previous run: longer parm_scalelen_vals

100m 1 --> 1

250m 2.9 --> 3.2

500m 4.5 --> 4.8

1000m 5.0 --> 5.3

#### Get the CESM tseries files for nutrients and other vars

In [ ]:
case = 'g.e21.G1850ECOIAF.T62_g16.003'

In [ ]:
path = '/glade/scratch/kristenk/archive/' + case + '/ocn/proc/tseries/month_1'

In [ ]:
variables = ['NO3', 'PO4','AOU','O2','SiO3']

In [ ]:
coords = {'x':'TLONG','y':'TLAT'}

In [ ]:
keepthese = ['z_t','z_t_150m','time_bound','TAREA','REGION_MASK', 'time'] + variables

In [ ]:
%%time

ds_list = []

print(f'reading {case}')

ds = xr.Dataset()

for v in variables:
    
    print(v)
    
    files = sorted(glob(f'{path}/{case}.pop.h.{v}.??????-??????.nc'))       

    #print(files)
    
    dsv = xr.open_mfdataset(files, decode_times=True, decode_coords=False)
    #dsv=xr.open_mfdataset(files, data_vars="minimal", coords='minimal', compat="override", parallel=True, concat_dim="time",
                       #drop_variables=["transport_components", "transport_regions"], decode_times=True)

    
    ds = xr.merge((ds, dsv))
    ds_list.append(ds)

#### Just keep the variables we need

In [ ]:
keep_vars = keepthese +list(coords.values())+['dz','KMT']
ds = ds.drop([v for v in ds.variables if v not in keep_vars])

In [ ]:
ds

#### Fix the time dimension so it's the mean of the time bounds

In [ ]:
ds["time"] = ds.time_bound.compute().mean(dim="d2")

#### Get the World Ocean Atlas dataset that has been interpolated onto the POP grid

In [ ]:
file = '/glade/work/kristenk/WOA_data/regrid_POP/WOA2013_POPgrid.nc'

In [ ]:
ds_woa = xr.load_dataset(file, decode_times=False, decode_coords=False)

#### reset the WOA depth dimension as it's slightly different from CESM one

In [ ]:
ds_woa['z_t'] = ds.z_t
tlen=len(ds.time)
tlen

#### Make annual mean variables for the last 10 years of the model simulation

In [ ]:
ds_annmean = xr.Dataset()
ds_annmean['NO3'] = ds.NO3.isel(time=slice(-120,tlen)).mean(dim='time')
ds_annmean['PO4'] = ds.PO4.isel(time=slice(-120,tlen)).mean(dim='time')
ds_annmean['SiO3'] = ds.SiO3.isel(time=slice(-120,tlen)).mean(dim='time')
ds_annmean['AOU'] = ds.AOU.isel(time=slice(-120,tlen)).mean(dim='time')
ds_annmean['O2'] = ds.O2.isel(time=slice(-120,tlen)).mean(dim='time')

#### Make difference arrays for the nutrients, O$_2$, and AOU

In [ ]:
NO3_diff = ds_annmean.NO3 - ds_woa.NO3
PO4_diff = ds_annmean.PO4 - ds_woa.PO4
SiO3_diff = ds_annmean.SiO3 - ds_woa.SiO3
AOU_diff = ds_annmean.AOU - ds_woa.AOU
O2_diff = ds_annmean.O2 - ds_woa.O2

#### Make an annual mean timeseries dataset

In [ ]:
%%time
ds_annmean_ts = ds.resample({'time':'A'}).mean(dim='time').compute()

#### Set up the longs and lons for plotting

In [ ]:
lons=ds_woa.TLONG
lats=ds_woa.TLAT
area=ds_woa.TAREA
depths=ds_woa.z_t * 0.01
years = np.arange(len(ds_annmean_ts.time))

#### Set up region mask for major basins

In [ ]:
grid_name = 'POP_gx1v7'
ds_regmasks = pop_tools.get_grid(grid_name)

In [ ]:
region_masks = pop_tools.list_region_masks(grid_name)
region_mask='Pacific-Indian-Atlantic'
mask3d = pop_tools.region_mask_3d(grid_name, mask_name=region_mask)

#### -------------------------------------------------------------------------- end of setup

### Surface macronutrient maps

In [ ]:
fig = plt.figure(figsize=(15,10))
plt.suptitle('Surface PO4 and NO3, mean over last 10 years of simulation', fontsize=14)

#COLUMN 1  - NO3
#---- CESM panel
ax = fig.add_subplot(3,2,1, projection=ccrs.PlateCarree())
ax.set_extent([-180, 180, -90, 90], ccrs.PlateCarree())
ax.coastlines('10m',linewidth=0.5)
ax.set_title('CESM annual mean surface NO$_3$', fontsize=10)
pc1=ax.pcolormesh(lons, lats, 
                 ds_annmean.NO3.isel(z_t=0), vmin=0, vmax=20, cmap='Oranges',
                 transform=ccrs.PlateCarree())
cbar1 = fig.colorbar(pc1, ax=ax,extend='max',label='NO$_3$ (mmol m$^{-3}$)')

#---- OBS panel
ax = fig.add_subplot(3,2,3, projection=ccrs.PlateCarree())
ax.set_extent([-180, 180, -90, 90], ccrs.PlateCarree())
ax.coastlines('10m',linewidth=0.5)
ax.set_title('WOA annual mean surface NO$_3$', fontsize=10)
pc2=ax.pcolormesh(lons, lats, 
                 ds_woa.NO3.isel(z_t=0), vmin=0, vmax=20, cmap='Oranges',
                 transform=ccrs.PlateCarree())
cbar1 = fig.colorbar(pc2, ax=ax,extend='max',label='NO$_3$ (mmol m$^{-3}$)')

#---- DIFF panel
ax = fig.add_subplot(3,2,5, projection=ccrs.PlateCarree())
ax.set_extent([-180, 180, -90, 90], ccrs.PlateCarree())
ax.coastlines('10m',linewidth=0.5)
ax.set_title('Surface NO$_3$ model bias', fontsize=10)
pc3=ax.pcolormesh(lons, lats, 
                 NO3_diff.isel(z_t=0), vmin=-10, vmax=10, cmap='bwr',
                 transform=ccrs.PlateCarree())
cbar1 = fig.colorbar(pc3, ax=ax,extend='both',label='NO$_3$ bias (mmol m$^{-3}$)')

#---- CESM panel
ax = fig.add_subplot(3,2,2, projection=ccrs.PlateCarree())
ax.set_extent([-180, 180, -90, 90], ccrs.PlateCarree())
ax.coastlines('10m',linewidth=0.5)
ax.set_title('CESM annual mean surface PO$_4$', fontsize=10)
pc1=ax.pcolormesh(lons, lats, 
                 ds_annmean.PO4.isel(z_t=0), vmin=0, vmax=2, cmap='Oranges',
                 transform=ccrs.PlateCarree())
cbar1 = fig.colorbar(pc1, ax=ax,extend='max',label='PO$_4$ (mmol m$^{-3}$)')

#---- OBS panel
ax = fig.add_subplot(3,2,4, projection=ccrs.PlateCarree())
ax.set_extent([-180, 180, -90, 90], ccrs.PlateCarree())
ax.coastlines('10m',linewidth=0.5)
ax.set_title('WOA annual mean surface PO$_4$', fontsize=10)
pc2=ax.pcolormesh(lons, lats, 
                 ds_woa.PO4.isel(z_t=0), vmin=0, vmax=2, cmap='Oranges',
                 transform=ccrs.PlateCarree())
cbar1 = fig.colorbar(pc2, ax=ax,extend='max',label='PO$_4$ (mmol m$^{-3}$)')

#---- DIFF panel
ax = fig.add_subplot(3,2,6, projection=ccrs.PlateCarree())
ax.set_extent([-180, 180, -90, 90], ccrs.PlateCarree())
ax.coastlines('10m',linewidth=0.5)
ax.set_title('Surface PO$_4$ bias', fontsize=10)
pc3=ax.pcolormesh(lons, lats, 
                 PO4_diff.isel(z_t=0), vmin=-1, vmax=1, cmap='bwr',
                 transform=ccrs.PlateCarree())
cbar1 = fig.colorbar(pc3, ax=ax,extend='both',label='PO$_4$ bias (mmol m$^{-3}$)')



### Thermocline macronutrient maps (~480m)

In [ ]:
fig = plt.figure(figsize=(15,10))

plt.suptitle('Thermocline PO4 and NO3, mean over last 10 years of simulation', fontsize=14)


#COLUMN 1  - NO3

#---- CESM panel
ax = fig.add_subplot(3,2,1, projection=ccrs.PlateCarree())
ax.set_extent([-180, 180, -90, 90], ccrs.PlateCarree())
ax.coastlines('10m',linewidth=0.5)
ax.set_title('CESM annual mean NO$_3$ at 480m', fontsize=10)
pc1=ax.pcolormesh(lons, lats, 
                 ds_annmean.NO3.isel(z_t=32), vmin=0, vmax=40, cmap='Oranges',
                 transform=ccrs.PlateCarree())
cbar1 = fig.colorbar(pc1, ax=ax,extend='max',label='NO$_3$ (mmol m$^{-3}$)')

#---- OBS panel
ax = fig.add_subplot(3,2,3, projection=ccrs.PlateCarree())
ax.set_extent([-180, 180, -90, 90], ccrs.PlateCarree())
ax.coastlines('10m',linewidth=0.5)
ax.set_title('WOA annual mean NO$_3$ at 480m', fontsize=10)
pc2=ax.pcolormesh(lons, lats, 
                 ds_woa.NO3.isel(z_t=32), vmin=0, vmax=40, cmap='Oranges',
                 transform=ccrs.PlateCarree())
cbar1 = fig.colorbar(pc2, ax=ax,extend='max',label='NO$_3$ (mmol m$^{-3}$)')

#---- DIFF panel
ax = fig.add_subplot(3,2,5, projection=ccrs.PlateCarree())
ax.set_extent([-180, 180, -90, 90], ccrs.PlateCarree())
ax.coastlines('10m',linewidth=0.5)
ax.set_title('NO$_3$ model bias at 480m', fontsize=10)
pc3=ax.pcolormesh(lons, lats, 
                 NO3_diff.isel(z_t=32), vmin=-15, vmax=15, cmap='bwr',
                 transform=ccrs.PlateCarree())
cbar1 = fig.colorbar(pc3, ax=ax,extend='both',label='NO$_3$ bias (mmol m$^{-3}$)')

#---- CESM panel
ax = fig.add_subplot(3,2,2, projection=ccrs.PlateCarree())
ax.set_extent([-180, 180, -90, 90], ccrs.PlateCarree())
ax.coastlines('10m',linewidth=0.5)
ax.set_title('CESM annual mean PO$_4$ at 480m', fontsize=10)
pc1=ax.pcolormesh(lons, lats, 
                 ds_annmean.PO4.isel(z_t=32), vmin=0, vmax=4, cmap='Oranges',
                 transform=ccrs.PlateCarree())
cbar1 = fig.colorbar(pc1, ax=ax,extend='max',label='NO$_3$ (mmol m$^{-3}$)')

#---- OBS panel
ax = fig.add_subplot(3,2,4, projection=ccrs.PlateCarree())
ax.set_extent([-180, 180, -90, 90], ccrs.PlateCarree())
ax.coastlines('10m',linewidth=0.5)
ax.set_title('WOA annual mean PO$_4$ at 480m', fontsize=10)
pc2=ax.pcolormesh(lons, lats, 
                 ds_woa.PO4.isel(z_t=32), vmin=0, vmax=4, cmap='Oranges',
                 transform=ccrs.PlateCarree())
cbar1 = fig.colorbar(pc2, ax=ax,extend='max',label='NO$_3$ (mmol m$^{-3}$)')

#---- DIFF panel
ax = fig.add_subplot(3,2,6, projection=ccrs.PlateCarree())
ax.set_extent([-180, 180, -90, 90], ccrs.PlateCarree())
ax.coastlines('10m',linewidth=0.5)
ax.set_title('PO$_4$ bias  at 480m', fontsize=10)
pc3=ax.pcolormesh(lons, lats, 
                 PO4_diff.isel(z_t=32), vmin=-1, vmax=1, cmap='bwr',
                 transform=ccrs.PlateCarree())
cbar1 = fig.colorbar(pc3, ax=ax,extend='both',label='PO$_4$ bias (mmol m$^{-3}$)')

### Thermocline O$_2$ and AOU maps

In [ ]:
fig = plt.figure(figsize=(15,10))

plt.suptitle('Thermocline O2 and AOU, mean over last 10 years of simulation', fontsize=14)

#COLUMN 1  - O2
#---- CESM panel
ax = fig.add_subplot(3,2,1, projection=ccrs.PlateCarree())
ax.set_extent([-180, 180, -90, 90], ccrs.PlateCarree())
ax.coastlines('10m',linewidth=0.5)
ax.set_title('CESM annual mean O$_2$ at 480m', fontsize=10)
pc1=ax.pcolormesh(lons, lats, 
                 ds_annmean.O2.isel(z_t=32),cmap='GnBu',
                 vmin=0, vmax=300, 
                 transform=ccrs.PlateCarree())
cbar1 = fig.colorbar(pc1, ax=ax,extend='max',label='O$_2$ (mmol m$^{-3}$)')

#---- OBS panel
ax = fig.add_subplot(3,2,3, projection=ccrs.PlateCarree())
ax.set_extent([-180, 180, -90, 90], ccrs.PlateCarree())
ax.coastlines('10m',linewidth=0.5)
ax.set_title('WOA annual mean O$_2$ at 480m', fontsize=10)
pc2=ax.pcolormesh(lons, lats, 
                 ds_woa.O2.isel(z_t=32), cmap='GnBu',
                 vmin=0, vmax=300, 
                 transform=ccrs.PlateCarree())
cbar1 = fig.colorbar(pc2, ax=ax,extend='max',label='O$_2$ (mmol m$^{-3}$)')

#---- DIFF panel
ax = fig.add_subplot(3,2,5, projection=ccrs.PlateCarree())
ax.set_extent([-180, 180, -90, 90], ccrs.PlateCarree())
ax.coastlines('10m',linewidth=0.5)
ax.set_title('O$_2$ at 480m model bias', fontsize=10)
pc3=ax.pcolormesh(lons, lats, 
                 O2_diff.isel(z_t=32), cmap='bwr',
                 vmin=-100, vmax=100, 
                 transform=ccrs.PlateCarree())
cbar1 = fig.colorbar(pc3, ax=ax,extend='both',label='O$_2$ bias (mmol m$^{-3}$)')

#column 2 AOU
#---- CESM panel
ax = fig.add_subplot(3,2,2, projection=ccrs.PlateCarree())
ax.set_extent([-180, 180, -90, 90], ccrs.PlateCarree())
ax.coastlines('10m',linewidth=0.5)
ax.set_title('CESM annual mean AOU at 480m', fontsize=10)
pc1=ax.pcolormesh(lons, lats, 
                 ds_woa.AOU.isel(z_t=32), cmap='BuPu',
                 vmin=0, vmax=300, 
                 transform=ccrs.PlateCarree())
cbar1 = fig.colorbar(pc1, ax=ax,extend='max',label='AOU  (mmol m$^{-3}$)')

#---- OBS panel
ax = fig.add_subplot(3,2,4, projection=ccrs.PlateCarree())
ax.set_extent([-180, 180, -90, 90], ccrs.PlateCarree())
ax.coastlines('10m',linewidth=0.5)
ax.set_title('WOA annual mean AOU at 480m', fontsize=10)
pc2=ax.pcolormesh(lons, lats, 
                 ds_woa.AOU.isel(z_t=32), cmap='BuPu',
                 vmin=0, vmax=300, 
                 transform=ccrs.PlateCarree())
cbar1 = fig.colorbar(pc2, ax=ax,extend='max',label='AOU (mmol m$^{-3}$)')

#---- DIFF panel
ax = fig.add_subplot(3,2,6, projection=ccrs.PlateCarree())
ax.set_extent([-180, 180, -90, 90], ccrs.PlateCarree())
ax.coastlines('10m',linewidth=0.5)
ax.set_title('AOU at 480m bias', fontsize=10)
pc3=ax.pcolormesh(lons, lats, 
                 AOU_diff.isel(z_t=32), cmap='bwr',
                vmin=-100, vmax=100, 
                 transform=ccrs.PlateCarree())
cbar1 = fig.colorbar(pc3, ax=ax,extend='both',label='AOU bias (mmol m$^{-3}$)')

#### Global mean nutrient profiles

In [ ]:
ds_glb = xr.Dataset({v: ds[v] for v in ['time', 'time_bound']})
ds_glb['NO3']= esmlab.weighted_mean(ds_annmean['NO3'], weights=ds.TAREA, dim=['nlat', 'nlon']).compute()
ds_glb['PO4']= esmlab.weighted_mean(ds_annmean['PO4'], weights=ds.TAREA, dim=['nlat', 'nlon']).compute()
ds_glb['SiO3']= esmlab.weighted_mean(ds_annmean['SiO3'], weights=ds.TAREA, dim=['nlat', 'nlon']).compute()
ds_glb['O2']= esmlab.weighted_mean(ds_annmean['O2'], weights=ds.TAREA, dim=['nlat', 'nlon']).compute()
ds_glb['AOU']= esmlab.weighted_mean(ds_annmean['AOU'], weights=ds.TAREA, dim=['nlat', 'nlon']).compute()

In [ ]:
ds_glb_woa = xr.Dataset({v: ds[v] for v in ['time', 'time_bound']})
ds_glb_woa['NO3'] = esmlab.weighted_mean(ds_woa['NO3'], weights=ds.TAREA, dim=['nlat', 'nlon']).compute()
ds_glb_woa['PO4'] = esmlab.weighted_mean(ds_woa['PO4'], weights=ds.TAREA, dim=['nlat', 'nlon']).compute()
ds_glb_woa['SiO3'] = esmlab.weighted_mean(ds_woa['SiO3'], weights=ds.TAREA, dim=['nlat', 'nlon']).compute()
ds_glb_woa['O2'] = esmlab.weighted_mean(ds_woa['O2'], weights=ds.TAREA, dim=['nlat', 'nlon']).compute()
ds_glb_woa['AOU'] = esmlab.weighted_mean(ds_woa['AOU'], weights=ds.TAREA, dim=['nlat', 'nlon']).compute()

In [ ]:
fig = plt.figure(figsize=(15,10))

plt.suptitle('Global mean macronutrient profiles, mean over last 10 years of simulation', fontsize=14)


#COLUMN 1  - NO3

ax = fig.add_subplot(2,3,1)
ax.set_title('Global mean NO$_3$')
ax.plot(ds_glb['NO3'].values, depths, label='CESM', linewidth=3)
ax.plot(ds_glb_woa['NO3'].values, depths, label='WOA', linewidth=3)
ax.legend()
ax.set(ylabel='depth (m)',xlabel='NO$_3$ (mmol m$^{-3}$)')
plt.gca().invert_yaxis()

#COLUMN 2  - PO4

ax = fig.add_subplot(2,3,2)
ax.set_title('Global mean PO$_4$')
ax.plot(ds_glb['PO4'].values, depths, label='CESM', linewidth=3)
ax.plot(ds_glb_woa['PO4'].values, depths, label='WOA', linewidth=3)
ax.legend()
ax.set(ylabel='depth (m)',xlabel='PO$_4$ (mmol m$^{-3}$)')
plt.gca().invert_yaxis()

#COLUMN 3  - SiO3

ax = fig.add_subplot(2,3,3)
ax.set_title('Global mean SiO$_3$')
ax.plot(ds_glb['SiO3'].values, depths, label='CESM', linewidth=3)
ax.plot(ds_glb_woa['SiO3'].values, depths, label='WOA', linewidth=3)
ax.legend()
ax.set(ylabel='depth (m)',xlabel='SiO$_3$ (mmol m$^{-3}$)')
plt.gca().invert_yaxis()


#COLUMN 1  - NO3 diff

ax = fig.add_subplot(2,3,4)
ax.plot(ds_glb['NO3'].values - ds_glb_woa['NO3'].values, depths, label='bias', linewidth=3, color='green')
ax.legend()
ax.set(ylabel='depth (m)',xlabel='NO$_3$ bias (mmol m$^{-3}$)')
plt.gca().invert_yaxis()

#COLUMN 2  - PO4 diff

ax = fig.add_subplot(2,3,5)
ax.plot(ds_glb['PO4'].values - ds_glb_woa['PO4'].values, depths, label='bias', linewidth=3, color='green')
ax.legend()
ax.set(ylabel='depth (m)',xlabel='PO$_4$ bias (mmol m$^{-3}$)')
plt.gca().invert_yaxis()

#COLUMN 3  - SiO3 diff

ax = fig.add_subplot(2,3,6)
ax.plot(ds_glb['SiO3'].values - ds_glb_woa['SiO3'].values, depths, label='bias', linewidth=3, color='green')
ax.legend()
ax.set(ylabel='depth (m)',xlabel='SiO$_3$ bias (mmol m$^{-3}$)')
plt.gca().invert_yaxis()



#### Just plotting the upper 500m

In [ ]:
fig = plt.figure(figsize=(16,5))

plt.suptitle('Global mean macronutrient upper ocean profiles, mean over last 10 years of simulation', fontsize=14)

#COLUMN 1  - NO3

ax = fig.add_subplot(1,3,1)
ax.set_title('Global mean NO$_3$')
ax.plot(ds_glb['NO3'].values, depths, label='CESM',linewidth=3)
ax.plot(ds_glb_woa['NO3'].values, depths, label='WOA',linewidth=3)
ax.legend()
ax.set_ylim(0,400)
ax.set_xlim(0,30)
ax.set(ylabel='depth (m)',xlabel='NO$_3$ (mmol m$^{-3}$)')
plt.gca().invert_yaxis()

#COLUMN 2  - PO4

ax = fig.add_subplot(1,3,2)
ax.set_title('Global mean PO$_4$')
ax.plot(ds_glb['PO4'].values, depths, label='CESM',linewidth=3)
ax.plot(ds_glb_woa['PO4'].values, depths, label='WOA',linewidth=3)
ax.legend()
ax.set_ylim(0,400)
ax.set_xlim(0,2.5)
ax.set(ylabel='depth (m)',xlabel='PO$_4$ (mmol m$^{-3}$)')
plt.gca().invert_yaxis()

#COLUMN 3  - SiO3

ax = fig.add_subplot(1,3,3)
ax.set_title('Global mean SiO$_3$')
ax.plot(ds_glb['SiO3'].values, depths, label='CESM',linewidth=3)
ax.plot(ds_glb_woa['SiO3'].values, depths, label='WOA',linewidth=3)
ax.legend()
ax.set_ylim(0,400)
ax.set_xlim(0,30)
ax.set(ylabel='depth (m)',xlabel='SiO$_3$ (mmol m$^{-3}$)')
plt.gca().invert_yaxis()

### Oxygen profile

In [ ]:
fig = plt.figure(figsize=(12,10))
plt.suptitle('Global mean O2 and AOU profiles, mean over last 10 years of simulation', fontsize=14)


#COLUMN 1  - O2

ax = fig.add_subplot(2,2,1)
ax.set_title('Global mean O$_2$')
ax.plot(ds_glb['O2'].values, depths, label='CESM',linewidth=3)
ax.plot(ds_glb_woa['O2'].values, depths, label='WOA',linewidth=3)
ax.legend()
ax.set(ylabel='depth (m)',xlabel='O$_2$ (mmol m$^{-3}$)')
plt.gca().invert_yaxis()


#COLUMN 2  - AOU

ax = fig.add_subplot(2,2,2)
ax.set_title('Global mean AOU')
ax.plot(ds_glb['AOU'].values, depths, label='CESM',linewidth=3)
ax.plot(ds_glb_woa['AOU'].values, depths, label='WOA',linewidth=3)
ax.legend()
ax.set(ylabel='depth (m)',xlabel='AOU (mmol m$^{-3}$)')
plt.gca().invert_yaxis()


#COLUMN 1  - O2

ax = fig.add_subplot(2,2,3)
ax.plot(ds_glb['O2'].values - ds_glb_woa['O2'].values, depths, label='bias',linewidth=3, color='green')
ax.legend()
ax.set(ylabel='depth (m)',xlabel='O$_2$ bias (mmol m$^{-3}$)')
plt.gca().invert_yaxis()


#COLUMN 2  - AOU

ax = fig.add_subplot(2,2,4)
ax.set_title('Global mean AOU')
ax.plot(ds_glb['AOU'].values - ds_glb_woa['AOU'].values, depths, label='bias',linewidth=3, color='green')
ax.legend()
ax.set(ylabel='depth (m)',xlabel='AOU bias (mmol m$^{-3}$)')
plt.gca().invert_yaxis()

### Regional nutrient profiles

#### Pacific

In [ ]:
regnum=2
mask3d.isel(region=regnum).plot();

In [ ]:
ds_pac = xr.Dataset({v: ds[v] for v in ['time', 'time_bound']})
ds_pac['NO3']= esmlab.weighted_mean(ds_annmean['NO3']*mask3d.isel(region=regnum), weights=ds.TAREA, dim=['nlat', 'nlon']).compute()
ds_pac['PO4']= esmlab.weighted_mean(ds_annmean['PO4']*mask3d.isel(region=regnum), weights=ds.TAREA, dim=['nlat', 'nlon']).compute()
ds_pac['SiO3']= esmlab.weighted_mean(ds_annmean['SiO3']*mask3d.isel(region=regnum), weights=ds.TAREA, dim=['nlat', 'nlon']).compute()
ds_pac['O2']= esmlab.weighted_mean(ds_annmean['O2']*mask3d.isel(region=regnum), weights=ds.TAREA, dim=['nlat', 'nlon']).compute()
ds_pac['AOU']= esmlab.weighted_mean(ds_annmean['AOU']*mask3d.isel(region=regnum), weights=ds.TAREA, dim=['nlat', 'nlon']).compute()

In [ ]:
ds_pac_woa = xr.Dataset({v: ds[v] for v in ['time', 'time_bound']})
ds_pac_woa['NO3'] = esmlab.weighted_mean(ds_woa['NO3']*mask3d.isel(region=regnum), weights=ds.TAREA, dim=['nlat', 'nlon']).compute()
ds_pac_woa['PO4'] = esmlab.weighted_mean(ds_woa['PO4']*mask3d.isel(region=regnum), weights=ds.TAREA, dim=['nlat', 'nlon']).compute()
ds_pac_woa['SiO3'] = esmlab.weighted_mean(ds_woa['SiO3']*mask3d.isel(region=regnum), weights=ds.TAREA, dim=['nlat', 'nlon']).compute()
ds_pac_woa['O2'] = esmlab.weighted_mean(ds_woa['O2']*mask3d.isel(region=regnum), weights=ds.TAREA, dim=['nlat', 'nlon']).compute()
ds_pac_woa['AOU'] = esmlab.weighted_mean(ds_woa['AOU']*mask3d.isel(region=regnum), weights=ds.TAREA, dim=['nlat', 'nlon']).compute()

In [ ]:
fig = plt.figure(figsize=(15,10))

#COLUMN 1  - NO3

ax = fig.add_subplot(2,3,1)
ax.set_title('Pacific basin mean NO$_3$')
ax.plot(ds_pac['NO3'].values, depths, label='CESM',linewidth=3)
ax.plot(ds_pac_woa['NO3'].values, depths, label='WOA',linewidth=3)
ax.legend()
ax.set(ylabel='depth (m)',xlabel='NO$_3$ (mmol m$^{-3}$)')
plt.gca().invert_yaxis()

#COLUMN 2  - PO4

ax = fig.add_subplot(2,3,2)
ax.set_title('Pacific basin mean PO$_4$')
ax.plot(ds_pac['PO4'].values, depths, label='CESM',linewidth=3)
ax.plot(ds_pac_woa['PO4'].values, depths, label='WOA',linewidth=3)
ax.legend()
ax.set(ylabel='depth (m)',xlabel='PO$_4$ (mmol m$^{-3}$)')
plt.gca().invert_yaxis()

#COLUMN 3  - SiO3

ax = fig.add_subplot(2,3,3)
ax.set_title('Pacific basin mean SiO$_3$')
ax.plot(ds_pac['SiO3'].values, depths, label='CESM',linewidth=3)
ax.plot(ds_pac_woa['SiO3'].values, depths, label='WOA',linewidth=3)
ax.legend()
ax.set(ylabel='depth (m)',xlabel='SiO$_3$ (mmol m$^{-3}$)')
plt.gca().invert_yaxis()


#COLUMN 1  - NO3 diff

ax = fig.add_subplot(2,3,4)
ax.plot(ds_pac['NO3'].values - ds_pac_woa['NO3'].values, depths, label='bias', linewidth=3, color='green')
ax.legend()
ax.set(ylabel='depth (m)',xlabel='NO$_3$ bias (mmol m$^{-3}$)')
plt.gca().invert_yaxis()

#COLUMN 2  - PO4 diff

ax = fig.add_subplot(2,3,5)
ax.plot(ds_pac['PO4'].values - ds_pac_woa['PO4'].values, depths, label='bias', linewidth=3, color='green')
ax.legend()
ax.set(ylabel='depth (m)',xlabel='PO$_4$ bias (mmol m$^{-3}$)')
plt.gca().invert_yaxis()

#COLUMN 3  - SiO3 diff

ax = fig.add_subplot(2,3,6)
ax.plot(ds_pac['SiO3'].values - ds_pac_woa['SiO3'].values, depths, label='bias', linewidth=3, color='green')
ax.legend()
ax.set(ylabel='depth (m)',xlabel='SiO$_3$ bias (mmol m$^{-3}$)')
plt.gca().invert_yaxis()

In [ ]:
fig = plt.figure(figsize=(12,10))

#COLUMN 1  - O2

ax = fig.add_subplot(2,2,1)
ax.set_title('Pacific basin mean O$_2$')
ax.plot(ds_pac['O2'].values, depths, label='CESM',linewidth=3)
ax.plot(ds_pac_woa['O2'].values, depths, label='WOA',linewidth=3)
ax.legend()
ax.set(ylabel='depth (m)',xlabel='O$_2$ (mmol m$^{-3}$)')
plt.gca().invert_yaxis()

#COLUMN 2  - AOU

ax = fig.add_subplot(2,2,2)
ax.set_title('Pacific basin mean AOU')
ax.plot(ds_pac['AOU'].values, depths, label='CESM',linewidth=3)
ax.plot(ds_pac_woa['AOU'].values, depths, label='WOA',linewidth=3)
ax.legend()
ax.set(ylabel='depth (m)',xlabel='AOU (mmol m$^{-3}$)')
plt.gca().invert_yaxis()


#COLUMN 1  - O2 bias

ax = fig.add_subplot(2,2,3)
ax.plot(ds_pac['O2'].values - ds_pac_woa['O2'].values, depths, label='bias',linewidth=3, color='green')
ax.legend()
ax.set(ylabel='depth (m)',xlabel='O$_2$ bias (mmol m$^{-3}$)')
plt.gca().invert_yaxis()

#COLUMN 2  - AOU bias

ax = fig.add_subplot(2,2,4)
ax.plot(ds_pac['AOU'].values - ds_pac_woa['AOU'].values, depths, label='bias',linewidth=3, color='green')
ax.legend()
ax.set(ylabel='depth (m)',xlabel='AOU bias (mmol m$^{-3}$)')
plt.gca().invert_yaxis()


#### Atlantic

In [ ]:
regnum=1
mask3d.isel(region=regnum).plot();

In [ ]:
ds_atl = xr.Dataset({v: ds[v] for v in ['time', 'time_bound']})
ds_atl['NO3']= esmlab.weighted_mean(ds_annmean['NO3']*mask3d.isel(region=regnum), weights=ds.TAREA, dim=['nlat', 'nlon']).compute()
ds_atl['PO4']= esmlab.weighted_mean(ds_annmean['PO4']*mask3d.isel(region=regnum), weights=ds.TAREA, dim=['nlat', 'nlon']).compute()
ds_atl['SiO3']= esmlab.weighted_mean(ds_annmean['SiO3']*mask3d.isel(region=regnum), weights=ds.TAREA, dim=['nlat', 'nlon']).compute()
ds_atl['O2']= esmlab.weighted_mean(ds_annmean['O2']*mask3d.isel(region=regnum), weights=ds.TAREA, dim=['nlat', 'nlon']).compute()
ds_atl['AOU']= esmlab.weighted_mean(ds_annmean['AOU']*mask3d.isel(region=regnum), weights=ds.TAREA, dim=['nlat', 'nlon']).compute()

In [ ]:
ds_atl_woa = xr.Dataset({v: ds[v] for v in ['time', 'time_bound']})
ds_atl_woa['NO3'] = esmlab.weighted_mean(ds_woa['NO3']*mask3d.isel(region=regnum), weights=ds.TAREA, dim=['nlat', 'nlon']).compute()
ds_atl_woa['PO4'] = esmlab.weighted_mean(ds_woa['PO4']*mask3d.isel(region=regnum), weights=ds.TAREA, dim=['nlat', 'nlon']).compute()
ds_atl_woa['SiO3'] = esmlab.weighted_mean(ds_woa['SiO3']*mask3d.isel(region=regnum), weights=ds.TAREA, dim=['nlat', 'nlon']).compute()
ds_atl_woa['O2'] = esmlab.weighted_mean(ds_woa['O2']*mask3d.isel(region=regnum), weights=ds.TAREA, dim=['nlat', 'nlon']).compute()
ds_atl_woa['AOU'] = esmlab.weighted_mean(ds_woa['AOU']*mask3d.isel(region=regnum), weights=ds.TAREA, dim=['nlat', 'nlon']).compute()

In [ ]:
fig = plt.figure(figsize=(15,10))

#COLUMN 1  - NO3
ax = fig.add_subplot(2,3,1)
ax.set_title('Atlantic basin mean NO$_3$')
ax.plot(ds_atl['NO3'].values, depths, label='CESM',linewidth=3)
ax.plot(ds_atl_woa['NO3'].values, depths, label='WOA',linewidth=3)
ax.legend()
ax.set(ylabel='depth (m)',xlabel='NO$_3$ (mmol m$^{-3}$)')
plt.gca().invert_yaxis()

#COLUMN 2  - PO4
ax = fig.add_subplot(2,3,2)
ax.set_title('Atlantic basin mean PO$_4$')
ax.plot(ds_atl['PO4'].values, depths, label='CESM',linewidth=3)
ax.plot(ds_atl_woa['PO4'].values, depths, label='WOA',linewidth=3)
ax.legend()
ax.set(ylabel='depth (m)',xlabel='PO$_4$ (mmol m$^{-3}$)')
plt.gca().invert_yaxis()

#COLUMN 3  - SiO3
ax = fig.add_subplot(2,3,3)
ax.set_title('Atlantic basin mean SiO$_3$')
ax.plot(ds_atl['SiO3'].values, depths, label='CESM',linewidth=3)
ax.plot(ds_atl_woa['SiO3'].values, depths, label='WOA',linewidth=3)
ax.legend()
ax.set(ylabel='depth (m)',xlabel='SiO$_3$ (mmol m$^{-3}$)')
plt.gca().invert_yaxis()


#COLUMN 1  - NO3 bias
ax = fig.add_subplot(2,3,4)
ax.plot(ds_atl['NO3'].values - ds_atl_woa['NO3'].values, depths, label='bias',linewidth=3, color='green')
ax.legend()
ax.set(ylabel='depth (m)',xlabel='NO$_3$ bias (mmol m$^{-3}$)')
plt.gca().invert_yaxis()

#COLUMN 2  - PO4 bias
ax = fig.add_subplot(2,3,5)
ax.plot(ds_atl['PO4'].values - ds_atl_woa['PO4'].values, depths, label='bias',linewidth=3, color='green')
ax.legend()
ax.set(ylabel='depth (m)',xlabel='PO$_4$ bias (mmol m$^{-3}$)')
plt.gca().invert_yaxis()

#COLUMN 3  - SiO3 bias
ax = fig.add_subplot(2,3,6)
ax.plot(ds_atl['SiO3'].values - ds_atl_woa['SiO3'].values, depths, label='bias',linewidth=3, color='green')
ax.legend()
ax.set(ylabel='depth (m)',xlabel='SiO$_3$ bias (mmol m$^{-3}$)')
plt.gca().invert_yaxis()

In [ ]:
fig = plt.figure(figsize=(12,10))

#COLUMN 1  - O2
ax = fig.add_subplot(2,2,1)
ax.set_title('Atlantic mean O$_2$')
ax.plot(ds_atl['O2'].values, depths, label='CESM',linewidth=3)
ax.plot(ds_atl_woa['O2'].values, depths, label='WOA',linewidth=3)
ax.legend()
ax.set(ylabel='depth (m)',xlabel='O$_2$ (mmol m$^{-3}$)')
plt.gca().invert_yaxis()

#COLUMN 1  - AOU
ax = fig.add_subplot(2,2,2)
ax.set_title('Atlantic mean AOU')
ax.plot(ds_atl['AOU'].values, depths, label='CESM',linewidth=3)
ax.plot(ds_atl_woa['AOU'].values, depths, label='WOA',linewidth=3)
ax.legend()
ax.set(ylabel='depth (m)',xlabel='AOU (mmol m$^{-3}$)')
plt.gca().invert_yaxis()


#COLUMN 1  - O2 bias
ax = fig.add_subplot(2,2,3)
ax.plot(ds_atl['O2'].values - ds_atl_woa['O2'].values, depths, label='CESM',linewidth=3, color='green')
ax.legend()
ax.set(ylabel='depth (m)',xlabel='O$_2$ bias (mmol m$^{-3}$)')
plt.gca().invert_yaxis()

#COLUMN 1  - AOU bias
ax = fig.add_subplot(2,2,4)
ax.plot(ds_atl['AOU'].values - ds_atl_woa['AOU'].values, depths, label='CESM',linewidth=3, color='green')
ax.legend()
ax.set(ylabel='depth (m)',xlabel='AOU bias (mmol m$^{-3}$)')
plt.gca().invert_yaxis()

### Look at drift in nutrients and O$_2$ at 500m

In [ ]:
ds_annmean_ts

#### Global

In [ ]:
%%time
ts_glob_500m = xr.Dataset({v: ds_annmean_ts[v] for v in ['time']})
ts_glob_500m['O2'] = esmlab.weighted_mean(ds_annmean_ts['O2'].isel(z_t=32), weights=area, dim=['nlat', 'nlon']).compute()
ts_glob_500m['PO4'] = esmlab.weighted_mean(ds_annmean_ts['PO4'].isel(z_t=32), weights=area, dim=['nlat', 'nlon']).compute()
ts_glob_500m['NO3'] = esmlab.weighted_mean(ds_annmean_ts['NO3'].isel(z_t=32), weights=area, dim=['nlat', 'nlon']).compute()
ts_glob_500m['SiO3'] = esmlab.weighted_mean(ds_annmean_ts['SiO3'].isel(z_t=32), weights=area, dim=['nlat', 'nlon']).compute()

#### Atlantic

In [ ]:
ts_Atl = xr.Dataset({v: ds_annmean_ts[v] for v in ['time']})
regnum=1
ts_Atl['O2'] = esmlab.weighted_mean(ds_annmean_ts['O2'].isel(z_t=32)*mask3d.isel(region=regnum), weights=area, dim=['nlat', 'nlon']).compute()
ts_Atl['PO4'] = esmlab.weighted_mean(ds_annmean_ts['PO4'].isel(z_t=32)*mask3d.isel(region=regnum), weights=area, dim=['nlat', 'nlon']).compute()
ts_Atl['NO3'] = esmlab.weighted_mean(ds_annmean_ts['NO3'].isel(z_t=32)*mask3d.isel(region=regnum), weights=area, dim=['nlat', 'nlon']).compute()
ts_Atl['SiO3'] = esmlab.weighted_mean(ds_annmean_ts['SiO3'].isel(z_t=32)*mask3d.isel(region=regnum), weights=area, dim=['nlat', 'nlon']).compute()

#### Pacific

In [ ]:
ts_Pac = xr.Dataset({v: ds_annmean_ts[v] for v in ['time']})
regnum=2
ts_Pac['O2'] = esmlab.weighted_mean(ds_annmean_ts['O2'].isel(z_t=32)*mask3d.isel(region=regnum), weights=area, dim=['nlat', 'nlon']).compute()
ts_Pac['PO4'] = esmlab.weighted_mean(ds_annmean_ts['PO4'].isel(z_t=32)*mask3d.isel(region=regnum), weights=area, dim=['nlat', 'nlon']).compute()
ts_Pac['NO3'] = esmlab.weighted_mean(ds_annmean_ts['NO3'].isel(z_t=32)*mask3d.isel(region=regnum), weights=area, dim=['nlat', 'nlon']).compute()
ts_Pac['SiO3'] = esmlab.weighted_mean(ds_annmean_ts['SiO3'].isel(z_t=32)*mask3d.isel(region=regnum), weights=area, dim=['nlat', 'nlon']).compute()

#### Indian

In [ ]:
ts_Ind = xr.Dataset({v: ds_annmean_ts[v] for v in ['time']})
regnum=3
ts_Ind['O2'] = esmlab.weighted_mean(ds_annmean_ts['O2'].isel(z_t=32)*mask3d.isel(region=regnum), weights=area, dim=['nlat', 'nlon']).compute()
ts_Ind['PO4'] = esmlab.weighted_mean(ds_annmean_ts['PO4'].isel(z_t=32)*mask3d.isel(region=regnum), weights=area, dim=['nlat', 'nlon']).compute()
ts_Ind['NO3'] = esmlab.weighted_mean(ds_annmean_ts['NO3'].isel(z_t=32)*mask3d.isel(region=regnum), weights=area, dim=['nlat', 'nlon']).compute()
ts_Ind['SiO3'] = esmlab.weighted_mean(ds_annmean_ts['SiO3'].isel(z_t=32)*mask3d.isel(region=regnum), weights=area, dim=['nlat', 'nlon']).compute()

### Plot O$_2$ timeseries for each basin

In [ ]:
fig = plt.figure(figsize=(16,5))

ax = fig.add_subplot(1,3,1)
ax.set_title('O$_2$ at 480m in the Atlantic')
ax.scatter(years,ts_Atl['O2'],color='blue')
ax.set(ylabel='O$_2$ (mmol m$^{-3}$)',xlabel='year of simulation')

ax = fig.add_subplot(1,3,2)
ax.set_title('O$_2$ at 480m in the Indian')
ax.scatter(years,ts_Ind['O2'],color='blue')
ax.set(ylabel='O$_2$ (mmol m$^{-3}$)',xlabel='year of simulation')

ax = fig.add_subplot(1,3,3)
ax.set_title('O$_2$ at 480m in the Pacific')
ax.scatter(years,ts_Pac['O2'],color='blue')
ax.set(ylabel='O$_2$ (mmol m$^{-3}$)',xlabel='year of simulation');

### Plot global O$_2$ timeseries

In [ ]:
fig = plt.figure(figsize=(16,5))

ax = fig.add_subplot(1,1,1)
ax.set_title('Global O$_2$ at 480m')
ax.scatter(years,ts_glob_500m['O2'],color='blue')
ax.set(ylabel='O$_2$ (mmol m$^{-3}$)',xlabel='year of simulation')

### Plot PO$_4$ timeseries for basins

In [ ]:
fig = plt.figure(figsize=(16,5))

ax = fig.add_subplot(1,3,1)
ax.set_title('PO$_4$ in the Atlantic')
ax.scatter(years,ts_Atl['PO4'],color='blue')
ax.set(ylabel='PO$_4$ (mmol m$^{-3}$)',xlabel='year of simulation')

ax = fig.add_subplot(1,3,2)
ax.set_title('PO$_4$ in the Indian')
ax.scatter(years,ts_Ind['PO4'],color='blue')
ax.set(ylabel='PO$_4$ (mmol m$^{-3}$)',xlabel='year of simulation')

ax = fig.add_subplot(1,3,3)
ax.set_title('PO$_4$ in the Pacific')
ax.scatter(years,ts_Pac['PO4'],color='blue')
ax.set(ylabel='PO$_4$ (mmol m$^{-3}$)',xlabel='year of simulation');

### Plot global PO$_4$ timeseries

In [ ]:
fig = plt.figure(figsize=(16,5))

ax = fig.add_subplot(1,1,1)
ax.set_title('Global PO$_4$ at 480m')
ax.scatter(years,ts_glob_500m['PO4'],color='blue')
ax.set(ylabel='PO$_4$ (mmol m$^{-3}$)',xlabel='year of simulation')

### Plot NO$_3$ timeseries for basins

In [ ]:
fig = plt.figure(figsize=(16,5))

ax = fig.add_subplot(1,3,1)
ax.set_title('NO$_3$ in the Atlantic')
ax.scatter(years,ts_Atl['NO3'],color='blue')
ax.set(ylabel='NO$_3$ (mmol m$^{-3}$)',xlabel='year of simulation')

ax = fig.add_subplot(1,3,2)
ax.set_title('NO$_3$ in the Indian')
ax.scatter(years,ts_Ind['NO3'],color='blue')
ax.set(ylabel='NO$_3$ (mmol m$^{-3}$)',xlabel='year of simulation')

ax = fig.add_subplot(1,3,3)
ax.set_title('NO$_3$ in the Pacific')
ax.scatter(years,ts_Pac['NO3'],color='blue')
ax.set(ylabel='NO$_3$ (mmol m$^{-3}$)',xlabel='year of simulation');

### Plot global NO$_3$ timeseries

In [ ]:
fig = plt.figure(figsize=(16,5))

ax = fig.add_subplot(1,1,1)
ax.set_title('Global NO$_3$ at 480m')
ax.scatter(years,ts_glob_500m['NO3'],color='blue')
ax.set(ylabel='NO$_3$ (mmol m$^{-3}$)',xlabel='year of simulation')

### Plot the SiO3 timeseries for each basin

In [ ]:
fig = plt.figure(figsize=(16,5))

ax = fig.add_subplot(1,3,1)
ax.set_title('SiO$_3$ in the Atlantic')
ax.scatter(years,ts_Atl['SiO3'],color='blue')
ax.set(ylabel='SiO$_3$ (mmol m$^{-3}$)',xlabel='year of simulation')

ax = fig.add_subplot(1,3,2)
ax.set_title('SiO$_3$ in the Indian')
ax.scatter(years,ts_Ind['SiO3'],color='blue')
ax.set(ylabel='SiO$_3$ (mmol m$^{-3}$)',xlabel='year of simulation')

ax = fig.add_subplot(1,3,3)
ax.set_title('SiO$_3$ in the Pacific')
ax.scatter(years,ts_Pac['SiO3'],color='blue')
ax.set(ylabel='SiO$_3$ (mmol m$^{-3}$)',xlabel='year of simulation');

### Plot the global SiO$_3$ timeseries

In [ ]:
fig = plt.figure(figsize=(16,5))

ax = fig.add_subplot(1,1,1)
ax.set_title('Global SiO$_3$ at 480m')
ax.scatter(years,ts_glob_500m['SiO3'],color='blue')
ax.set(ylabel='SiO$_3$ (mmol m$^{-3}$)',xlabel='year of simulation')

## Hovmoller diagrams of nutient/O2 biases (depth versus time)

#### WOA for making biases

In [ ]:
woa_Pac = xr.Dataset({v: ds_woa[v] for v in ['z_t']})
regnum=2
woa_Pac['O2'] = esmlab.weighted_mean(ds_woa['O2']*mask3d.isel(region=regnum), weights=area, dim=['nlat', 'nlon']).compute()
woa_Pac['NO3'] = esmlab.weighted_mean(ds_woa['NO3']*mask3d.isel(region=regnum), weights=area, dim=['nlat', 'nlon']).compute()
woa_Pac['PO4'] = esmlab.weighted_mean(ds_woa['PO4']*mask3d.isel(region=regnum), weights=area, dim=['nlat', 'nlon']).compute()
woa_Pac['AOU'] = esmlab.weighted_mean(ds_woa['AOU']*mask3d.isel(region=regnum), weights=area, dim=['nlat', 'nlon']).compute()
woa_Pac['SiO3'] = esmlab.weighted_mean(ds_woa['SiO3']*mask3d.isel(region=regnum), weights=area, dim=['nlat', 'nlon']).compute()

In [ ]:
woa_Atl = xr.Dataset({v: ds_woa[v] for v in ['z_t']})
regnum=1
woa_Atl['O2'] = esmlab.weighted_mean(ds_woa['O2']*mask3d.isel(region=regnum), weights=area, dim=['nlat', 'nlon']).compute()
woa_Atl['NO3'] = esmlab.weighted_mean(ds_woa['NO3']*mask3d.isel(region=regnum), weights=area, dim=['nlat', 'nlon']).compute()
woa_Atl['PO4'] = esmlab.weighted_mean(ds_woa['PO4']*mask3d.isel(region=regnum), weights=area, dim=['nlat', 'nlon']).compute()
woa_Atl['AOU'] = esmlab.weighted_mean(ds_woa['AOU']*mask3d.isel(region=regnum), weights=area, dim=['nlat', 'nlon']).compute()
woa_Atl['SiO3'] = esmlab.weighted_mean(ds_woa['SiO3']*mask3d.isel(region=regnum), weights=area, dim=['nlat', 'nlon']).compute()

In [ ]:
woa_global = xr.Dataset({v: ds_woa[v] for v in ['z_t']})
woa_global['O2'] = esmlab.weighted_mean(ds_woa['O2'], weights=area, dim=['nlat', 'nlon']).compute()
woa_global['NO3'] = esmlab.weighted_mean(ds_woa['NO3'], weights=area, dim=['nlat', 'nlon']).compute()
woa_global['PO4'] = esmlab.weighted_mean(ds_woa['PO4'], weights=area, dim=['nlat', 'nlon']).compute()
woa_global['AOU'] = esmlab.weighted_mean(ds_woa['AOU'], weights=area, dim=['nlat', 'nlon']).compute()
woa_global['SiO3'] = esmlab.weighted_mean(ds_woa['SiO3'], weights=area, dim=['nlat', 'nlon']).compute()

#### CESM timeseries: Pacific, Alantic, global

Pacific

In [ ]:
%%time
regnum=2
ts_Pac = xr.Dataset({v: ds_annmean_ts[v] for v in ['time','z_t']})
ts_Pac['O2'] = esmlab.weighted_mean(ds_annmean_ts['O2']*mask3d.isel(region=regnum), weights=area, dim=['nlat', 'nlon']).compute()
ts_Pac['PO4'] = esmlab.weighted_mean(ds_annmean_ts['PO4']*mask3d.isel(region=regnum), weights=area, dim=['nlat', 'nlon']).compute()
ts_Pac['NO3'] = esmlab.weighted_mean(ds_annmean_ts['NO3']*mask3d.isel(region=regnum), weights=area, dim=['nlat', 'nlon']).compute()
ts_Pac['AOU'] = esmlab.weighted_mean(ds_annmean_ts['AOU']*mask3d.isel(region=regnum), weights=area, dim=['nlat', 'nlon']).compute()
ts_Pac['SiO3'] = esmlab.weighted_mean(ds_annmean_ts['SiO3']*mask3d.isel(region=regnum), weights=area, dim=['nlat', 'nlon']).compute()

In [ ]:
diff_Pac = ts_Pac - woa_Pac
diff_Pac=diff_Pac.transpose()
diff_Pac_top=diff_Pac.isel(z_t=slice(0,40))
diff_Pac_bottom=diff_Pac.isel(z_t=slice(40,60))

Atlantic

In [ ]:
%%time
regnum=1
ts_Atl = xr.Dataset({v: ds_annmean_ts[v] for v in ['time','z_t']})
ts_Atl['O2'] = esmlab.weighted_mean(ds_annmean_ts['O2']*mask3d.isel(region=regnum), weights=area, dim=['nlat', 'nlon']).compute()
ts_Atl['PO4'] = esmlab.weighted_mean(ds_annmean_ts['PO4']*mask3d.isel(region=regnum), weights=area, dim=['nlat', 'nlon']).compute()
ts_Atl['NO3'] = esmlab.weighted_mean(ds_annmean_ts['NO3']*mask3d.isel(region=regnum), weights=area, dim=['nlat', 'nlon']).compute()
ts_Atl['AOU'] = esmlab.weighted_mean(ds_annmean_ts['AOU']*mask3d.isel(region=regnum), weights=area, dim=['nlat', 'nlon']).compute()
ts_Atl['SiO3'] = esmlab.weighted_mean(ds_annmean_ts['SiO3']*mask3d.isel(region=regnum), weights=area, dim=['nlat', 'nlon']).compute()

In [ ]:
diff_Atl = ts_Atl - woa_Atl
diff_Atl=diff_Atl.transpose()
diff_Atl_top=diff_Atl.isel(z_t=slice(0,40))
diff_Atl_bottom=diff_Atl.isel(z_t=slice(40,60))

Global

In [ ]:
%%time
ts_global = xr.Dataset({v: ds_annmean_ts[v] for v in ['time','z_t']})
ts_global['O2'] = esmlab.weighted_mean(ds_annmean_ts['O2'], weights=area, dim=['nlat', 'nlon']).compute()
ts_global['PO4'] = esmlab.weighted_mean(ds_annmean_ts['PO4'], weights=area, dim=['nlat', 'nlon']).compute()
ts_global['NO3'] = esmlab.weighted_mean(ds_annmean_ts['NO3'], weights=area, dim=['nlat', 'nlon']).compute()
ts_global['AOU'] = esmlab.weighted_mean(ds_annmean_ts['AOU'], weights=area, dim=['nlat', 'nlon']).compute()
ts_global['SiO3'] = esmlab.weighted_mean(ds_annmean_ts['SiO3'], weights=area, dim=['nlat', 'nlon']).compute()

In [ ]:
diff_global = ts_global - woa_global
diff_global=diff_global.transpose()
diff_global_top=diff_global.isel(z_t=slice(0,40))
diff_global_bottom=diff_global.isel(z_t=slice(40,60))

#### Set up dims for plotting

In [ ]:
z_top=depths.isel(z_t=slice(0,40))
z_bottom=depths.isel(z_t=slice(40,60))

## O$_2$

#### Plot Pacific O$_2$ bias

In [ ]:
fig = plt.figure(figsize=(13,6))

plt.suptitle('Pacific O$_2$', fontsize=14)

ax = fig.add_subplot(2,1,1)
levels = [-14,-12,-10,-8,-6,-4,-2,0,2,4,6,8,10,12,14]
pc=ax.contourf(years, z_top, diff_Pac_top['O2'], levels, cmap='bwr')
contour = plt.contour(years, z_top, diff_Pac_top['O2'], levels, colors='k')
plt.clabel(contour, colors = 'k', fmt = '%2.0f', fontsize=12)
ax.set(ylabel='depth (m)') #, xlabel='time (simulation years)')
ax.invert_yaxis()
ax.xaxis.tick_top()

ax = fig.add_subplot(2,1,2)
pc=ax.contourf(years, z_bottom, diff_Pac_bottom['O2'], levels, cmap='bwr')
contour = plt.contour(years, z_bottom, diff_Pac_bottom['O2'], levels, colors='k')
plt.clabel(contour, colors = 'k', fmt = '%2.0f', fontsize=12)
ax.set(ylabel='depth (m)', xlabel='time (simulation years)')
ax.invert_yaxis()

fig.subplots_adjust(right=0.8,hspace=0)
cbar_ax = fig.add_axes([0.85, 0.15, 0.02, 0.7])
cbar = fig.colorbar(pc, cax=cbar_ax, label='O$_2$ (mmol/m3)')

In [ ]:
# #this cell plots normal depth versus time plot for Pacific O2
# fig = plt.figure(figsize=(15,7))
# plt.suptitle('Pacific', fontsize=12)
# ax = fig.add_subplot(1,1,1)
# ax.set_title('AOU', fontsize=10)
# levels = [-10, -8,-6,-4, -2, 0, 2, 4,6,8,10]
# pc=ax.contourf(years, depths, diff_Pac['O2'], levels, cmap='bwr')
# contour = plt.contour(years, depths, diff_Pac['O2'], levels, colors='k')
# plt.clabel(contour, colors = 'k', fmt = '%2.0f', fontsize=12)
# cbar1 = fig.colorbar(pc, ax=ax,extend='both',label='AOU (mmol/m3)')
# ax.set(ylabel='depth (m)', xlabel='time (simulation years)')
# ax.invert_yaxis()

#### Plot Atlantic O2 bias

In [ ]:
fig = plt.figure(figsize=(13,6))

plt.suptitle('Atlantic O$_2$', fontsize=14)

ax = fig.add_subplot(2,1,1)
levels = [-12,-10,-8,-6,-4,-2,0,2,4,6,8,10,12]
pc=ax.contourf(years, z_top, diff_Atl_top['O2'], levels, cmap='bwr')
contour = plt.contour(years, z_top, diff_Atl_top['O2'], levels, colors='k')
plt.clabel(contour, colors = 'k', fmt = '%2.0f', fontsize=12)
ax.set(ylabel='depth (m)') #, xlabel='time (simulation years)')
ax.invert_yaxis()
ax.xaxis.tick_top()

ax = fig.add_subplot(2,1,2)
pc=ax.contourf(years, z_bottom, diff_Atl_bottom['O2'], levels, cmap='bwr')
contour = plt.contour(years, z_bottom, diff_Atl_bottom['O2'], levels, colors='k')
plt.clabel(contour, colors = 'k', fmt = '%2.0f', fontsize=12)
ax.set(ylabel='depth (m)', xlabel='time (simulation years)')
ax.invert_yaxis()

fig.subplots_adjust(right=0.8,hspace=0)
cbar_ax = fig.add_axes([0.85, 0.15, 0.02, 0.7])
cbar = fig.colorbar(pc, cax=cbar_ax, label='O$_2$ (mmol/m3)')

#### Plot Global O2 bias

In [ ]:
fig = plt.figure(figsize=(13,6))

plt.suptitle('Global O$_2$', fontsize=14)

ax = fig.add_subplot(2,1,1)
levels = [-26,-24,-22,-20,-18,-16,-14,-12,-10,-8,-6,-4,-2,0,2,4,6,8,10,12,14,16,18,20,22,24,26]
pc=ax.contourf(years, z_top, diff_global_top['O2'], levels, cmap='bwr')
contour = plt.contour(years, z_top, diff_global_top['O2'], levels, colors='k')
plt.clabel(contour, colors = 'k', fmt = '%2.0f', fontsize=12)
ax.set(ylabel='depth (m)') #, xlabel='time (simulation years)')
ax.invert_yaxis()
ax.xaxis.tick_top()

ax = fig.add_subplot(2,1,2)
pc=ax.contourf(years, z_bottom, diff_global_bottom['O2'], levels, cmap='bwr')
contour = plt.contour(years, z_bottom, diff_global_bottom['O2'], levels, colors='k')
plt.clabel(contour, colors = 'k', fmt = '%2.0f', fontsize=12)
ax.set(ylabel='depth (m)', xlabel='time (simulation years)')
ax.invert_yaxis()

fig.subplots_adjust(right=0.8,hspace=0)
cbar_ax = fig.add_axes([0.85, 0.15, 0.02, 0.7])
cbar = fig.colorbar(pc, cax=cbar_ax, label='O$_2$ (mmol/m3)')

## NO$_3$

#### Plot Pacific NO$_3$ bias

In [ ]:
fig = plt.figure(figsize=(13,6))

plt.suptitle('Pacific NO$_3$', fontsize=14)

ax = fig.add_subplot(2,1,1)
levels = [-1.4,-1.2,-1,-0.8,-0.6,-0.4,-0.2,0,0.2,0.4,0.6,0.8,1,1.2,1.4]
pc=ax.contourf(years, z_top, diff_Pac_top['NO3'], levels, cmap='bwr')
contour = plt.contour(years, z_top, diff_Pac_top['NO3'], levels, colors='k')
plt.clabel(contour, colors = 'k', fmt = '%2.1f', fontsize=12)
ax.set(ylabel='depth (m)') #, xlabel='time (simulation years)')
ax.invert_yaxis()
ax.xaxis.tick_top()

ax = fig.add_subplot(2,1,2)
pc=ax.contourf(years, z_bottom, diff_Pac_bottom['NO3'], levels, cmap='bwr')
contour = plt.contour(years, z_bottom, diff_Pac_bottom['NO3'], levels, colors='k')
plt.clabel(contour, colors = 'k', fmt = '%2.0f', fontsize=12)
ax.set(ylabel='depth (m)', xlabel='time (simulation years)')
ax.invert_yaxis()

fig.subplots_adjust(right=0.8,hspace=0)
cbar_ax = fig.add_axes([0.85, 0.15, 0.02, 0.7])
cbar = fig.colorbar(pc, cax=cbar_ax, label='NO$_3$ (mmol/m3)')

#### Plot Atlantic NO$_3$ bias

In [ ]:
fig = plt.figure(figsize=(13,6))

plt.suptitle('Atlantic NO$_3$', fontsize=14)

ax = fig.add_subplot(2,1,1)
levels = [-1,-0.8,-0.6,-0.4,-0.2,0,0.2,0.4,0.6,0.8,1]
pc=ax.contourf(years, z_top, diff_Atl_top['NO3'], levels, cmap='bwr')
contour = plt.contour(years, z_top, diff_Atl_top['NO3'], levels, colors='k')
plt.clabel(contour, colors = 'k', fmt = '%2.1f', fontsize=12)
ax.set(ylabel='depth (m)') #, xlabel='time (simulation years)')
ax.invert_yaxis()
ax.xaxis.tick_top()

ax = fig.add_subplot(2,1,2)
pc=ax.contourf(years, z_bottom, diff_Atl_bottom['NO3'], levels, cmap='bwr')
contour = plt.contour(years, z_bottom, diff_Atl_bottom['NO3'], levels, colors='k')
plt.clabel(contour, colors = 'k', fmt = '%2.0f', fontsize=12)
ax.set(ylabel='depth (m)', xlabel='time (simulation years)')
ax.invert_yaxis()

fig.subplots_adjust(right=0.8,hspace=0)
cbar_ax = fig.add_axes([0.85, 0.15, 0.02, 0.7])
cbar = fig.colorbar(pc, cax=cbar_ax, label='NO$_3$ (mmol/m3)')

In [ ]:
#this cell plots normal depth versus time plot for Atlantic NO3 cause the above one looks a bit weird between upper and lower panel
fig = plt.figure(figsize=(15,7))
plt.suptitle('Atlantic', fontsize=12)
ax = fig.add_subplot(1,1,1)
ax.set_title('O$_2$', fontsize=10)
levels = [-1,-0.8,-0.6,-0.4,-0.2,0,0.2,0.4,0.6,0.8,1]
pc=ax.contourf(years, depths, diff_Atl['NO3'], levels, cmap='bwr')
contour = plt.contour(years, depths, diff_Atl['NO3'], levels, colors='k')
plt.clabel(contour, colors = 'k', fmt = '%2.0f', fontsize=12)
cbar1 = fig.colorbar(pc, ax=ax,extend='both',label='O2 (mmol/m3)')
ax.set(ylabel='depth (m)', xlabel='time (simulation years)')
ax.invert_yaxis()

#### Global NO$_3$ bias

In [ ]:
fig = plt.figure(figsize=(13,6))

plt.suptitle('Global NO$_3$', fontsize=14)

ax = fig.add_subplot(2,1,1)
levels = [-1.6,-1.4,-1.2,-1,-0.8,-0.6,-0.4,-0.2,0,0.2,0.4,0.6,0.8,1,1.2,1.4,1.6]
pc=ax.contourf(years, z_top, diff_global_top['NO3'], levels, cmap='bwr')
contour = plt.contour(years, z_top, diff_global_top['NO3'], levels, colors='k')
plt.clabel(contour, colors = 'k', fmt = '%2.1f', fontsize=12)
ax.set(ylabel='depth (m)') #, xlabel='time (simulation years)')
ax.invert_yaxis()
ax.xaxis.tick_top()

ax = fig.add_subplot(2,1,2)
pc=ax.contourf(years, z_bottom, diff_global_bottom['NO3'], levels, cmap='bwr')
contour = plt.contour(years, z_bottom, diff_global_bottom['NO3'], levels, colors='k')
plt.clabel(contour, colors = 'k', fmt = '%2.0f', fontsize=12)
ax.set(ylabel='depth (m)', xlabel='time (simulation years)')
ax.invert_yaxis()

fig.subplots_adjust(right=0.8,hspace=0)
cbar_ax = fig.add_axes([0.85, 0.15, 0.02, 0.7])
cbar = fig.colorbar(pc, cax=cbar_ax, label='NO$_3$ (mmol/m3)')

## SiO$_3$

#### Atlantic SiO$_3$ bias

In [ ]:
fig = plt.figure(figsize=(13,6))

plt.suptitle('Atlantic SiO$_3$', fontsize=14)

ax = fig.add_subplot(2,1,1)
levels = [-4, -3,-2,-1.5,-1,-0.5,0,0.5,1,1.5,2,3,4]
pc=ax.contourf(years, z_top, diff_Atl_top['SiO3'], levels, cmap='bwr')
contour = plt.contour(years, z_top, diff_Atl_top['SiO3'], levels, colors='k')
plt.clabel(contour, colors = 'k', fmt = '%2.1f', fontsize=12)
ax.set(ylabel='depth (m)') #, xlabel='time (simulation years)')
ax.invert_yaxis()
ax.xaxis.tick_top()

ax = fig.add_subplot(2,1,2)
pc=ax.contourf(years, z_bottom, diff_Atl_bottom['SiO3'], levels, cmap='bwr')
contour = plt.contour(years, z_bottom, diff_Atl_bottom['SiO3'], levels, colors='k')
plt.clabel(contour, colors = 'k', fmt = '%2.0f', fontsize=12)
ax.set(ylabel='depth (m)', xlabel='time (simulation years)')
ax.invert_yaxis()

fig.subplots_adjust(right=0.8,hspace=0)
cbar_ax = fig.add_axes([0.85, 0.15, 0.02, 0.7])
cbar = fig.colorbar(pc, cax=cbar_ax, label='SiO$_3$ (mmol/m3)')

#### Pacific SiO$_3$ bias

In [ ]:
fig = plt.figure(figsize=(13,6))

plt.suptitle('Pacific SiO$_3$', fontsize=14)

ax = fig.add_subplot(2,1,1)
levels = [-4, -3,-2,-1.5,-1,-0.5,0,0.5,1,1.5,2,3,4]
pc=ax.contourf(years, z_top, diff_Pac_top['SiO3'], levels, cmap='bwr')
contour = plt.contour(years, z_top, diff_Pac_top['SiO3'], levels, colors='k')
plt.clabel(contour, colors = 'k', fmt = '%2.1f', fontsize=12)
ax.set(ylabel='depth (m)') #, xlabel='time (simulation years)')
ax.invert_yaxis()
ax.xaxis.tick_top()

ax = fig.add_subplot(2,1,2)
pc=ax.contourf(years, z_bottom, diff_Pac_bottom['SiO3'], levels, cmap='bwr')
contour = plt.contour(years, z_bottom, diff_Pac_bottom['SiO3'], levels, colors='k')
plt.clabel(contour, colors = 'k', fmt = '%2.0f', fontsize=12)
ax.set(ylabel='depth (m)', xlabel='time (simulation years)')
ax.invert_yaxis()

fig.subplots_adjust(right=0.8,hspace=0)
cbar_ax = fig.add_axes([0.85, 0.15, 0.02, 0.7])
cbar = fig.colorbar(pc, cax=cbar_ax, label='SiO$_3$ (mmol/m3)')

#### Global SiO$_3$ bias

In [ ]:
fig = plt.figure(figsize=(13,6))

plt.suptitle('Global SiO$_3$', fontsize=14)

ax = fig.add_subplot(2,1,1)
levels = [-7,-6,-5,-4,-3,-2,-1,0,1,2,3,4,5,6,7]
pc=ax.contourf(years, z_top, diff_global_top['SiO3'], levels, cmap='bwr')
contour = plt.contour(years, z_top, diff_global_top['SiO3'], levels, colors='k')
plt.clabel(contour, colors = 'k', fmt = '%2.1f', fontsize=12)
ax.set(ylabel='depth (m)') #, xlabel='time (simulation years)')
ax.invert_yaxis()
ax.xaxis.tick_top()

ax = fig.add_subplot(2,1,2)
pc=ax.contourf(years, z_bottom, diff_global_bottom['SiO3'], levels, cmap='bwr')
contour = plt.contour(years, z_bottom, diff_global_bottom['SiO3'], levels, colors='k')
plt.clabel(contour, colors = 'k', fmt = '%2.0f', fontsize=12)
ax.set(ylabel='depth (m)', xlabel='time (simulation years)')
ax.invert_yaxis()

fig.subplots_adjust(right=0.8,hspace=0)
cbar_ax = fig.add_axes([0.85, 0.15, 0.02, 0.7])
cbar = fig.colorbar(pc, cax=cbar_ax, label='SiO$_3$ (mmol/m3)')

## PO$_4$

#### Atlantic PO$_4$ bias

In [ ]:
fig = plt.figure(figsize=(13,6))

plt.suptitle('Atlantic PO$_4$', fontsize=14)

ax = fig.add_subplot(2,1,1)
levels = [-0.05,-0.04,-0.03,-0.02,-0.01,0,0.01,0.02,0.03,0.04,0.05]
pc=ax.contourf(years, z_top, diff_Atl_top['PO4'], levels, cmap='bwr')
contour = plt.contour(years, z_top, diff_Atl_top['PO4'], levels, colors='k')
plt.clabel(contour, colors = 'k', fmt = '%2.1f', fontsize=12)
ax.set(ylabel='depth (m)') #, xlabel='time (simulation years)')
ax.invert_yaxis()
ax.xaxis.tick_top()

ax = fig.add_subplot(2,1,2)
pc=ax.contourf(years, z_bottom, diff_Atl_bottom['PO4'], levels, cmap='bwr')
contour = plt.contour(years, z_bottom, diff_Atl_bottom['PO4'], levels, colors='k')
plt.clabel(contour, colors = 'k', fmt = '%2.0f', fontsize=12)
ax.set(ylabel='depth (m)', xlabel='time (simulation years)')
ax.invert_yaxis()

fig.subplots_adjust(right=0.8,hspace=0)
cbar_ax = fig.add_axes([0.85, 0.15, 0.02, 0.7])
cbar = fig.colorbar(pc, cax=cbar_ax, label='PO$_4$ (mmol/m3)')

#### Pacific PO$_4$ bias

In [ ]:
fig = plt.figure(figsize=(13,6))

plt.suptitle('Pacific PO$_4$', fontsize=14)

ax = fig.add_subplot(2,1,1)
levels = [-0.06,-0.05,-0.04,-0.03,-0.02,-0.01,0,0.01,0.02,0.03,0.04,0.05,0.06]
pc=ax.contourf(years, z_top, diff_Pac_top['PO4'], levels, cmap='bwr')
contour = plt.contour(years, z_top, diff_Pac_top['PO4'], levels, colors='k')
plt.clabel(contour, colors = 'k', fmt = '%2.1f', fontsize=12)
ax.set(ylabel='depth (m)') #, xlabel='time (simulation years)')
ax.invert_yaxis()
ax.xaxis.tick_top()

ax = fig.add_subplot(2,1,2)
pc=ax.contourf(years, z_bottom, diff_Pac_bottom['PO4'], levels, cmap='bwr')
contour = plt.contour(years, z_bottom, diff_Pac_bottom['PO4'], levels, colors='k')
plt.clabel(contour, colors = 'k', fmt = '%2.0f', fontsize=12)
ax.set(ylabel='depth (m)', xlabel='time (simulation years)')
ax.invert_yaxis()

fig.subplots_adjust(right=0.8,hspace=0)
cbar_ax = fig.add_axes([0.85, 0.15, 0.02, 0.7])
cbar = fig.colorbar(pc, cax=cbar_ax, label='PO$_4$ (mmol/m3)')

#### Global PO$_4$ bias

In [ ]:
fig = plt.figure(figsize=(13,6))

plt.suptitle('Global PO$_4$', fontsize=14)

ax = fig.add_subplot(2,1,1)
levels = [-0.11,-0.1,-0.09,-0.08,-0.07,-0.06,-0.05,-0.04,-0.03,-0.02,-0.01,0,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1,0.11]
pc=ax.contourf(years, z_top, diff_global_top['PO4'], levels, cmap='bwr')
contour = plt.contour(years, z_top, diff_global_top['PO4'], levels, colors='k')
plt.clabel(contour, colors = 'k', fmt = '%2.1f', fontsize=12)
ax.set(ylabel='depth (m)') #, xlabel='time (simulation years)')
ax.invert_yaxis()
ax.xaxis.tick_top()

ax = fig.add_subplot(2,1,2)
pc=ax.contourf(years, z_bottom, diff_global_bottom['PO4'], levels, cmap='bwr')
contour = plt.contour(years, z_bottom, diff_global_bottom['PO4'], levels, colors='k')
plt.clabel(contour, colors = 'k', fmt = '%2.0f', fontsize=12)
ax.set(ylabel='depth (m)', xlabel='time (simulation years)')
ax.invert_yaxis()

fig.subplots_adjust(right=0.8,hspace=0)
cbar_ax = fig.add_axes([0.85, 0.15, 0.02, 0.7])
cbar = fig.colorbar(pc, cax=cbar_ax, label='PO$_4$ (mmol/m3)')